# Starter
1. 資料格式壓縮(減少記憶體使用)
2. to_feather(fast_loading)

# EDA 資料探勘

- 缺少值探勘

In [ ]:
msno.matrix()

- 視覺化 Sns

In [ ]:
# 多變量圖(pairplot)
sns.pairplot(df['col1'],df['col2'],df['col3'],df['col4'])
# 直方圖(含KDE)(distplot)
sns.distplot(df['col1'])
# 純KDE
sns.kdeplot(df['col1'])
# 視覺化風格
plt.style.use('ggplot')
# 直接用散狀圖幫你畫出線性迴歸趨勢! Great!
sns.regplot(x="total_bill", y="tip", data=tips);

- ECDF

In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF 
#ECDF：累積分佈函數
cdf = ECDF(df["特定資料函位"])
plt.plot(cdf.x, cdf.y)

# 特徵工程(資料前處理)

## 編碼
> 處理物件或非數值欄位
1. 標籤編碼(有序離散值) 請用Label Encoder
2. 無序離散值用One Hot Encoder(小心特徵爆炸)
3. 二值化(label_binarize)
4. 計數編碼(Counting) 跟筆數有正相關，例如交易幾筆
5. 均值編碼(Mean Encoding) - 與target有關，就用該類的avg target編碼
> 注意 由於是用target的data leak，很容易overfitting，資料筆數過少時，容易被極端值影響，務必CV
6. 雜湊編碼
7. 群聚編碼: 是特徵彼此之間的augment(標籤->數值)mean, count等等去編碼
> 群聚編碼因為與目標值無關，因此不容易 Overfitting
8. 基於樹狀的葉編碼(LEAF ENCODING)
> 先跑過一次模型用樹的分類當作特徵，有點BOOSTING的味道

In [ ]:
# Deal with Objects
from sklearn.preprocessing import LabelEncoder
# Label Encoder
le = LabelEncoder()
le.fit(app_train[col])
app_train[col] = le.transform(app_train[col])
# One Hot Encoder
#get_dummies : 僅能將字串轉換為One hot encoding表示形式， 沒指定columns會全部轉換。
app_train = pd.get_dummies(app_train)

## 增加編碼(feature augment)
- 時間 - 常用(日/月/周/假日)
- 地點 - 經緯度

In [ ]:
df['pickup_month'] = df['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%m')).astype('int64')

## 離散化
> 提升計算效率，降低outlier影響
- 等寬、等頻
- kmeans/二值化

In [ ]:
ages = pd.DataFrame({"age": [18,22,25,27,7,21,23,37,30,61,45,41,9,18,80,100]})
ages["equal_width_age"] = pd.cut(ages["age"], 4) # 等寬劃分
ages["equal_freq_age"] = pd.qcut(ages["age"], 4) # 等頻劃分
ages.head(5)

- 補缺值(平均/中位數或NaN)

In [ ]:
from sklearn.preprocessing import Imputer
# 填補器 : 設定缺失值補中位數
imputer = Imputer(strategy = 'median')
# 填補器載入個欄中位數
imputer.fit(train)
# 將中位數回填 train, test 資料中的空缺值
train = imputer.transform(train)


# 無量綱化
- 歸一化-區間縮放或標準化[參考](https://medium.com/ai%E5%8F%8D%E6%96%97%E5%9F%8E/preprocessing-data-%E6%95%B8%E6%93%9A%E7%89%B9%E5%BE%B5%E6%A8%99%E6%BA%96%E5%8C%96%E5%92%8C%E6%AD%B8%E4%B8%80%E5%8C%96-9bd3e5a8f2fc)
- 加速訓練，特別是深度學習前，必須歸一化，否則很難收斂(橢圓與圓形的梯度下降)
- 使各特徵均衡(不被平方差大的主導)
- 歸一化要注意，會被異常值影響(例如很大的Outlier): 應用 影像辨識/255
- 標準化大多時間較好(符合統計+高斯分布 無量綱)
- 幾乎都要做歸一或標準化 情境如下
1. 有L1 L2正則化
2. 有做PCA
3. SVM、 SGD、linear/logistic regression 、Kmeans、KNN、NN（如果有用L1/L2 regularization）涉及到距離有關的算法
- DecisionTree 、0/1取值的特徵、 基於平方損失的最小二乘法OLS

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# 歸一化物件導向
scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(train)
train = scaler.transform(train)

# Normalization = MinMaxScaler
(value - min(value) ) / ( max(value) - min(value) )

# Standardization z-score 標準化 
value - np.mean(value) / ( np.std(value) )
def z_transform(x):
    x = (x - np.mean(x)) / np.std(x)
    return x

## 處理Large Skewness的方法
- 注意np.log1p的使用，使target可以符合標準分布
- log1p = log（x+1） 保證x很小時，還是不為0，反運算exmp1()
- boxcox也是一種方式
- 去除極端Outlier有助於訓練過程

In [ ]:
#離群值限制範圍
df['GrLivArea'] = df['GrLivArea'].clip(500, 2000)
#直接把離群值丟掉
# 使用布林函數

# 特徵篩選
- [知乎神文](https://www.zhihu.com/question/28641663/answer/110165221)
1. 過濾法 (Filter) : 
- 相關係數過濾法(篩feature與target的相關係數)
- 變異數過濾法(篩去小變異的
2. 包裝法 (Wrapper) : 根據目標函數，逐步加入特徵或刪除特徵
> 使用預測模型給特徵子集打分。每個新子集都被用來訓練一個模型，然後用驗證數據集來測試。通過計算驗證數據集上的錯誤次數（即模型的錯誤率）給特徵子集評分。由於包裝類方法為每個特徵子集訓練一個新模型，所以計算量很大。不過，這類方法往往能為特定類型的模型找到性能最好的特徵集。


3. 嵌入法 (Embedded) : 使用機器學習模型，根據擬合後的係數，刪除係數低於門檻的特徵
- 基於逞罰: L1(Lasso)嵌入法: 調整正規化程度(逞罰) 讓某些擬合參數變為0，刪除0的
- 基於模型: GDBT(梯度提升樹)嵌入法: 

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from scipy.stats import pearsonr
#####1#####
#變異數
VarianceThreshold(threshold=3).fit_transform(iris.data)
#相關係數
SelectKBest(lambda X, Y: array(map(lambda x:pearsonr(x, Y), X.T)).T, k=2).fit_transform(iris.data, iris.target)
#卡方檢驗 - 檢查特徵與目標的相關性 - 計算不相干的值與實際值之平方差 HO假設
SelectKBest(chi2, k=2).fit_transform(iris.data, iris.target)

#####2#####
RFE(estimator=LogisticRegression(), n_features_to_select=2).fit_transform(iris.data, iris.target)

#####3#####
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
#L1
SelectFromModel(LogisticRegression(penalty="l1", C=0.1)).fit_transform(iris.data, iris.target)
from sklearn.feature_selection import SelectFromModel
#GBDT
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
SelectFromModel(GradientBoostingClassifier()).fit_transform(iris.data, iris.target)

- 用梯度提升樹去套模型 選.feature_importances
- 可利用特徵組合方法，把最重要的幾個特徵加減乘除做運算

In [ ]:
# 梯度提升樹擬合後, 將結果依照重要性由高到低排序
estimator = GradientBoostingRegressor()
estimator.fit(df.values, train_Y)
# estimator.feature_importances_ 就是模型的特徵重要性, 這邊先與欄位名稱結合起來, 才能看到重要性與欄位名稱的對照表
feats = pd.Series(data=estimator.feature_importances_, index=df.columns)

# 切分資料
- train_split
- kfold

# 訓練模型

- LASSO, Ridge Regression
- Lasso 為 Linear Regression 加上 L1，Ridge 為 Linear Regression 加上 L2，

In [ ]:
from sklearn.linear_model import LinearRegression #線性迴歸
from sklearn.linear_model import LogisticRegression #分類問題


# 設定模型與模型參數
estimator = LogisticRegression(C = 0.0001)
# 使用 Train 資料訓練模型
estimator.fit(train, train_labels)
pred = estimator.predict(test_X)
log_reg_pred = estimator.predict_proba(test)[:, 1]


# 評估模型

1. K-fold CV
- 交叉拆成K份，每次留一份當驗證集

In [ ]:
from sklearn.model_selection import cross_val_score
knn = KNeighborsClassifier(n_neighbors=10)
scores = cross_val_score(knn,X,y,cv=5,scoring='accuracy')

# 損失函數

- 回歸常用的損失函數: 均方誤差(Mean square error，MSE)和平均絕對值誤差(Mean absolute error，MAE)。分類問題常用的損失函數: 交叉熵(cross-entropy)
- 分類常用損失函數：
- AUC
- F1-Score (Precision, Recall)